In [1]:
import torch
from torch import nn
from transformers import AutoTokenizer

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

Device: cuda


In [3]:
# Configuração do Tokenizador
checkpoint = "bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

In [4]:
dados_treino = [
    "para fazer um suco de laranja use o espremedor",
    "a salada de alface leva tomate e cenoura picada",
    "o espinafre refogado fica ótimo com muito alho",
    "suco de morango com leite vira uma vitamina",
    "mousse de maracujá é uma sobremesa muito azeda",
    "brócolis ao forno combina com azeite de oliva",
    "salada de frutas precisa de banana e maçã fresca",
    "para preparar um suco de limão use um bom espremedor",
    "a salada de rúcula leva tomate e cenoura ralada",
    "o espinafre refogado combina bem com alho dourado",
    "suco de banana com leite vira uma vitamina doce",
    "mousse de limão é uma sobremesa bem cítrica",
    "brócolis assado combina com azeite e sal",
    "salada de frutas leva banana e maçã madura",
    "para fazer suco natural de laranja use espremedor manual",
    "a salada verde leva alface tomate e cenoura crua",
    "espinafre refogado fica saboroso com alho e cebola",
    "suco de abacate com leite vira uma vitamina cremosa",
    "mousse de maracujá é uma sobremesa refrescante",
    "brócolis no forno fica ótimo com azeite de oliva",
    "salada de frutas precisa de banana maçã e uva"
]

In [5]:
class DecoderLM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, dropout_prob=0.2):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        self.emb_dropout = nn.Dropout(dropout_prob)
        
        decoder_layer = nn.TransformerEncoderLayer(
            d_model=embedding_dim,
            nhead=8,
            dim_feedforward=256,
            dropout=dropout_prob, 
            batch_first=True
        )
        self.transformer = nn.TransformerEncoder(decoder_layer, num_layers=3)
        
        
        self.out_dropout = nn.Dropout(dropout_prob)
        self.fc = nn.Linear(embedding_dim, vocab_size)

    def forward(self, input_ids):
        sz = input_ids.size(1)
        mask = torch.triu(torch.ones(sz, sz) * float('-inf'), diagonal=1).to(device)
        
        x = self.embedding(input_ids)
        x = self.emb_dropout(x) 
        
        x = self.transformer(x, mask=mask)
        
        x = self.out_dropout(x) 
        return self.fc(x)

In [6]:
modelo = DecoderLM(tokenizer.vocab_size, 128).to(device)
otimizador = torch.optim.Adam(modelo.parameters(), lr=0.0001)
loss_function = nn.CrossEntropyLoss()

In [7]:
print("Iniciando treinamento...")

for epoch in range(300):
    running_loss = 0.0
    
    for frase in dados_treino:
        otimizador.zero_grad()
        
        # Tokeniza a frase
        tokens = tokenizer(frase, return_tensors="pt").to(device)
        ids = tokens['input_ids'] # Ex: [1, 2, 3, 4]
        
        # Input: todos os tokens exceto o último
        # Target: todos os tokens exceto o primeiro (deslocados)
        input_ids = ids[:, :-1]
        target_ids = ids[:, 1:]
        
        logits = modelo(input_ids)
        
        # O CrossEntropy espera [Batch * Seq, Vocab] e o Target [Batch * Seq]
        loss = loss_function(logits.reshape(-1, tokenizer.vocab_size), target_ids.reshape(-1))
        
        loss.backward()
        otimizador.step()
        running_loss += loss.item()

    if epoch % 20 == 0:
        print("training epoch: ", epoch)
        print("loss:", running_loss/len(dados_treino))

print("Treinamento concluído!")

Iniciando treinamento...
training epoch:  0
loss: 11.772491818382626
training epoch:  20
loss: 5.1436761901492165
training epoch:  40
loss: 2.984841846284412
training epoch:  60
loss: 1.8159056731632777
training epoch:  80
loss: 1.1687177760260445
training epoch:  100
loss: 0.8193212634041196
training epoch:  120
loss: 0.6245558105763935
training epoch:  140
loss: 0.49941526424317134
training epoch:  160
loss: 0.44743964359873817
training epoch:  180
loss: 0.39860242747125174
training epoch:  200
loss: 0.3382021841548738
training epoch:  220
loss: 0.3204441176993506
training epoch:  240
loss: 0.29311483672686983
training epoch:  260
loss: 0.26791830573763165
training epoch:  280
loss: 0.2722061886673882
Treinamento concluído!


In [8]:
def gerar_texto(inicio, max_tokens=10):
    modelo.eval()
    input_ids = tokenizer(inicio, return_tensors="pt")['input_ids'].to(device)
    
    for _ in range(max_tokens):
        with torch.no_grad():
            output = modelo(input_ids)
            # Pegamos o logit do último token previsto
            next_token_id = output[:, -1, :].argmax(dim=-1).unsqueeze(0)
            
            # Adicionamos o novo token à sequência (auto-regressivo)
            input_ids = torch.cat([input_ids, next_token_id], dim=1)
            
            if next_token_id.item() == tokenizer.sep_token_id:
                break
                
    return tokenizer.decode(input_ids[0], skip_special_tokens=True)

# Teste prático
print(f"Input: 'suco de' -> Output: '{gerar_texto('suco de')}'")
print(f"Input: 'a salada' -> Output: '{gerar_texto('a salada')}'")

Input: 'suco de' -> Output: 'suco de com leite vira uma vitamina doce'
Input: 'a salada' -> Output: 'a salada leva alface tomate e cenoura c'
